In [1]:
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from selenium import webdriver
import pandas as pd
import time

def login_and_scrape_data(fullname, username, password):
    chromedriver_path = "chromedriver.exe"
    
    # Use the ChromeOptions to set the executable_path
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument(f"executable_path={chromedriver_path}")
    
    with webdriver.Chrome(options=chrome_options) as driver:
        driver.get('https://phongdaotao2.ntt.edu.vn/sinh-vien-dang-nhap.html')
        time.sleep(5)
    # Find necessary elements
        find_account_input = driver.find_element(By.ID, "UserName")
        find_password_input = driver.find_element(By.ID, "Password")
        find_captcha_input = driver.find_element(By.ID, "Captcha")
        signin_button = driver.find_element(By.CLASS_NAME, "form-button")
 
    # Enter username and password
        find_account_input.clear()
        find_password_input.clear()

        find_account_input.send_keys(username)
        find_password_input.send_keys(password)

        find_captcha_input.clear()
        captcha_input = input('Nhập captcha: ').strip()
        find_captcha_input.send_keys(captcha_input)
        signin_button.click()

    # Redirect to attendance page and click AccordingAll button
        driver.get('https://phongdaotao2.ntt.edu.vn/chuong-trinh-khung.html')
        find_AccordionAll_button = driver.find_element(By.ID, "OpenAccordionAll")
        find_AccordionAll_button.click()


    # Get column names
        find_heading_element = driver.find_element(By.XPATH, '//*[@id="contnet"]/div/div[2]/div/div/div[2]/div/table/thead')
        heading_elements = find_heading_element.find_elements(By.TAG_NAME, 'th')
    # Get data
        tr_values = []
        tbodys = driver.find_elements(By.TAG_NAME, 'tbody')[:-1]
        for tbody in tbodys:
            trs = tbody.find_elements(By.TAG_NAME, 'tr')
            for tr in trs[2:]:  # Bỏ qua tr[1] và tr[2]
                tds = tr.find_elements(By.TAG_NAME, 'td')
                row_data = [td.text for td in tds]
                tr_values.append(row_data)

        df = pd.DataFrame(tr_values)
        return df

result_df1 = login_and_scrape_data('Cao Đăng Khoa','2000001001','thuytrang14092002')
result_df1


NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=124.0.6367.202)
Stacktrace:
	GetHandleVerifier [0x00007FF7BAF21562+60802]
	(No symbol) [0x00007FF7BAE9AC62]
	(No symbol) [0x00007FF7BAD57CE4]
	(No symbol) [0x00007FF7BAD2DFDF]
	(No symbol) [0x00007FF7BADD1E57]
	(No symbol) [0x00007FF7BADE98D1]
	(No symbol) [0x00007FF7BADCA923]
	(No symbol) [0x00007FF7BAD98FEC]
	(No symbol) [0x00007FF7BAD99C21]
	GetHandleVerifier [0x00007FF7BB2241FD+3217949]
	GetHandleVerifier [0x00007FF7BB266197+3488183]
	GetHandleVerifier [0x00007FF7BB25F11F+3459391]
	GetHandleVerifier [0x00007FF7BAFDB926+823622]
	(No symbol) [0x00007FF7BAEA5FFF]
	(No symbol) [0x00007FF7BAEA0F24]
	(No symbol) [0x00007FF7BAEA10B2]
	(No symbol) [0x00007FF7BAE91904]
	BaseThreadInitThunk [0x00007FFC1DC8257D+29]
	RtlUserThreadStart [0x00007FFC1F9AAA48+40]


In [ ]:
result = result_df1.iloc[:64, 1:7]
# Xóa các dòng có giá trị None
result_none = result[result.isna().all(axis=1)].index
result = result.drop(result_none)
result = result.drop(56, axis = 0)
result.columns = ['course name', 'course id', 'course', 'credit', 'Theoretical session', 'Lab session']
result['total sessions'] = result['Theoretical session'].astype('int') + result['Lab session'].astype('int')
result.to_csv('../../data/reformatted_data/curriculum_framework.csv', index=False)

In [ ]:
result.head(4)

,course name,course id,course,credit,Theoretical session,Lab session,total sessions
0,Kỹ năng giao tiếp,1001070006,,2,30,0,30
1,Dịch vụ Hành chính,1001074618,,3,45,0,45
2,Công nghệ thông tin,1001074673,,3,0,90,90
3,Tiếng Anh 1,1001074730,,6,60,60,120
